<a href="https://colab.research.google.com/github/NatalieGergov/Fatigue_Detection/blob/main/Sample_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [63]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
# Load the data
features_df = pd.read_pickle('/content/drive/MyDrive/facemesh_marker_coordinates.pkl')

In [66]:
# Include the fps for each video
import cv2

unique_videos = df['Video_ID'].unique()

fps_map = {}
for video_id in unique_videos:
    path = f"/content/drive/MyDrive/DROZY/videos_i8/{video_id}.mp4"
    cap = cv2.VideoCapture(path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    fps_map[video_id] = fps

features_df['fps'] = features_df['Video_ID'].map(fps_map)

In [67]:
# Compute average eyelid distance and normalize

features_df["eyelid_distance"] = (features_df["Left_Eyelid"] + features_df["Right_Eyelid"]) / 2

features_df["Individual_ID"] = features_df["Video_ID"].str.split("-").str[0]

features_df["eyelid_norm"] = features_df.groupby("Individual_ID")["eyelid_distance"].transform(
    lambda x: (x - x.mean()) / x.std()
)

In [68]:
# Functions to extract blink features

from scipy.signal import find_peaks

def detect_blinks(signal, fps, threshold=-0.7, min_gap=0.2):
    inverted = -signal
    peaks, _ = find_peaks(inverted, height=abs(threshold), distance=fps * min_gap)
    return peaks

# Mark all frames in each blink event as 1
def mark_blink_frames(signal, peaks, threshold=-0.7):
    blink_mask = np.zeros(len(signal), dtype=int)
    for peak in peaks:
        # Expand around peak until signal rises above threshold
        start = peak
        while start > 0 and signal[start] < threshold:
            start -= 1
        end = peak
        while end < len(signal) and signal[end] < threshold:
            end += 1
        blink_mask[start:end] = 1
    return blink_mask

def blink_count(blink_mask):
    return pd.Series(blink_mask).diff().eq(1).sum()

def blinking_ratio(blink_mask, fps):
    return blink_count(blink_mask) / (len(blink_mask) / fps)

def blink_durations(blink_mask, fps):
    durations = []
    count = 0
    for val in blink_mask:
        if val == 1:
            count += 1
        elif count > 0:
            durations.append(count / fps)
            count = 0
    if count > 0:
        durations.append(count / fps)
    return np.mean(durations) if durations else 0, np.max(durations) if durations else 0

def inter_blink_intervals(blink_mask, fps):
    starts = np.where(np.diff(blink_mask) == 1)[0]
    ends = np.where(np.diff(blink_mask) == -1)[0]

    ibi = []
    for i in range(1, len(starts)):
        ibi.append((starts[i] - ends[i-1]) / fps)
    return np.mean(ibi) if ibi else 0

In [69]:
# Create dataframe with blink features

video_metrics = []

for vid, sub_df in features_df.groupby("Video_ID"):
    fps = sub_df["fps"].iloc[0]  # fps is constant per video
    signal = sub_df["eyelid_norm"].values

    peaks = detect_blinks(signal, fps, threshold=-0.7, min_gap=0.2)
    blink_mask = mark_blink_frames(signal, peaks, threshold=-0.7)

    count = blink_count(blink_mask)
    ratio = blinking_ratio(blink_mask, fps)
    avg_dur, max_dur = blink_durations(blink_mask, fps)
    avg_ibi = inter_blink_intervals(blink_mask, fps)

    video_metrics.append({
        "Video_ID": vid,
        "blink_count": count,
        "blink_ratio": ratio,
        "avg_blink_duration": avg_dur,
        "max_blink_duration": max_dur,
        "avg_inter_blink_interval": avg_ibi
    })

metrics_df = pd.DataFrame(video_metrics)
metrics_df.head()

,Video_ID,blink_count,blink_ratio,avg_blink_duration,max_blink_duration,avg_inter_blink_interval
0,1-1,407,0.683459,0.160033,1.200000,1.299507
1,1-2,335,0.529114,0.390647,4.666667,1.487226
2,1-3,362,0.612452,0.630939,15.200000,0.992059
3,10-1,355,0.596104,0.237653,0.666667,1.440772
4,10-3,500,0.838504,0.217166,0.800000,2.168136


In [70]:
# Includes blink mask for original dataframe

features_df["blink"] = 0  # initialize column

for vid, sub_df in features_df.groupby("Video_ID"):
    signal = sub_df["eyelid_norm"].values
    fps = sub_df["fps"].iloc[0]

    # Detect peaks and mark blink frames
    peaks = detect_blinks(signal, fps, threshold=-0.7, min_gap=0.2)
    blink_mask = mark_blink_frames(signal, peaks, threshold=-0.7)

    # Assign blink mask back to df
    features_df.loc[sub_df.index, "blink"] = blink_mask

features_df.head()

,Video_ID,Frame,Left_EAR,Right_EAR,EAR,MAR,Left_Eyelid,Right_Eyelid,KSS_Score,fps,eyelid_distance,Individual_ID,eyelid_norm,blink
0,1-1,1,NaN,NaN,NaN,NaN,NaN,NaN,3,30.0,NaN,1,NaN,0
1,1-1,2,NaN,NaN,NaN,NaN,NaN,NaN,3,30.0,NaN,1,NaN,0
2,1-1,3,NaN,NaN,NaN,NaN,NaN,NaN,3,30.0,NaN,1,NaN,0
3,1-1,4,0.199663,0.329171,0.264417,0.457820,2.170239,5.567490,3,30.0,3.868865,1,-1.042796,0
4,1-1,5,0.355450,0.311738,0.333594,0.458974,5.495173,4.760164,3,30.0,5.127669,1,0.370060,0


In [71]:
def closure_percentage_blink(df):
    results = []
    for vid, sub_df in df.groupby("Video_ID"):
        total_frames = len(sub_df)
        closed_frames = (sub_df["blink"] == 1).sum()
        closure_pct = (closed_frames / total_frames) * 100 if total_frames > 0 else 0
        results.append({"Video_ID": vid, "closure_pct_blink": closure_pct})
    return pd.DataFrame(results)

closure_blink = closure_percentage_blink(features_df)
metrics_df = pd.merge(metrics_df, closure_blink, on="Video_ID", how="left")
metrics_df.head()

,Video_ID,blink_count,blink_ratio,avg_blink_duration,max_blink_duration,avg_inter_blink_interval,closure_pct_blink
0,1-1,407,0.683459,0.160033,1.200000,1.299507,10.937587
1,1-2,335,0.529114,0.390647,4.666667,1.487226,20.669685
2,1-3,362,0.612452,0.630939,15.200000,0.992059,38.642003
3,10-1,355,0.596104,0.237653,0.666667,1.440772,14.166573
4,10-3,500,0.838504,0.217166,0.800000,2.168136,18.245849


In [86]:
def count_yawns(video_df, fps_col='fps', mar_col='MAR', frame_col='Frame',
                dur_sec=1.0, gap_sec=0.7, dip_tolerance_frames=5,
                threshold_factor=2.0):

    fps = video_df[fps_col].iloc[0]
    mar = video_df[mar_col].values
    frames = video_df[frame_col].values

    # Threshold
    thr = np.mean(mar) + threshold_factor * np.std(mar)

    above = mar > thr

    filled = above.copy()
    gap = 0
    for i in range(len(above)):
        if not above[i]:
            gap += 1
            if gap <= dip_tolerance_frames and i > 0 and i < len(above)-1 and above[i-1] and above[i+1]:
                filled[i] = True
            else:
                gap = 0

    # Find continuous segments
    min_frames = int(dur_sec * fps)
    merge_gap = int(gap_sec * fps)

    events = []
    start = None
    for i, val in enumerate(filled):
        if val and start is None:
            start = i
        elif not val and start is not None:
            end = i - 1
            if end - start + 1 >= min_frames:
                events.append((frames[start], frames[end]))
            start = None
    if start is not None:
        end = len(filled) - 1
        if end - start + 1 >= min_frames:
            events.append((frames[start], frames[end]))

    # Merge events if close
    merged = []
    for event in events:
        if not merged:
            merged.append(event)
        else:
            prev_start, prev_end = merged[-1]
            if event[0] - prev_end <= merge_gap:
                merged[-1] = (prev_start, event[1])
            else:
                merged.append(event)

    return len(merged), merged, thr

# Function to detect the yawns for every video with unique video ID
def detect_yawns(df, kss_col='KSS_Score', video_col='Video_ID'):

    results = []

    # Group by each video
    for vid, group in df.groupby(video_col):
        count, events, thr = count_yawns(group)

        kss_val = group[kss_col].iloc[0] if kss_col in group.columns else None

        results.append({
            'Video_ID': vid,
            'KSS_Score': kss_val,
            'Yawn_Count': count,
            'Yawn_Events': events,
            'Threshold': thr
        })

    return pd.DataFrame(results)

In [73]:
# Assuming df has ['Video_ID', 'Frame', 'MAR', 'fps', 'KSS']
results_df = detect_yawns(features_df)

metrics_df = metrics_df.merge(
    results_df[['Video_ID', 'Yawn_Count']],
    on='Video_ID',
    how='left'
)

In [74]:
# Include kss score for each video
kss_per_video = features_df.groupby("Video_ID")["KSS_Score"].first().reset_index()
metrics_df = metrics_df.merge(kss_per_video, on="Video_ID", how="left")

In [75]:
# Standardize features by user
metrics_df['Individual'] = metrics_df['Video_ID'].str.split('-').str[0]

feature_cols = ["blink_count", "blink_ratio", "avg_blink_duration",
                "max_blink_duration", "avg_inter_blink_interval", "closure_pct_blink", "Yawn_Count"]

# Apply user-based z-score standardization (per Individual)
metrics_df_norm = metrics_df.copy()
metrics_df_norm[feature_cols] = metrics_df.groupby('Individual')[feature_cols].transform(
    lambda x: (x - x.mean()) / x.std()
)

In [76]:
metrics_df_norm.head()

,Video_ID,blink_count,blink_ratio,avg_blink_duration,max_blink_duration,avg_inter_blink_interval,closure_pct_blink,Yawn_Count,KSS_Score,Individual
0,1-1,1.072222,0.972337,-0.993079,-0.798528,0.159650,-0.887860,-0.577350,3,1
1,1-2,-0.907265,-1.025539,-0.013701,-0.323069,0.910571,-0.195428,-0.577350,6,1
2,1-3,-0.164957,0.053203,1.006780,1.121597,-1.070221,1.083288,1.154701,7,1
3,10-1,-0.707107,-0.707107,0.707107,-0.707107,-0.707107,-0.707107,-0.707107,3,10
4,10-3,0.707107,0.707107,-0.707107,0.707107,0.707107,0.707107,0.707107,7,10


### Classifier

In [77]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

In [78]:
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42)
}

In [79]:
# Include hyperparameter tuning
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [80]:
features = ['blink_count', 'blink_ratio', 'avg_blink_duration',
            'max_blink_duration', 'avg_inter_blink_interval', 'closure_pct_blink']

X = metrics_df_norm[features]
y = (metrics_df_norm['KSS_Score'] > 5).astype(int)  # binary classification: alert vs drowsy

In [81]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(random_state=42))
])

In [82]:
param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [None, 5, 10],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

In [83]:
grid = GridSearchCV(pipe, param_grid,
                    cv=5, scoring='f1', n_jobs=-1)

grid.fit(X, y)

print("Best parameters:", grid.best_params_)
print("Best F1 score (CV):", grid.best_score_)

Best parameters: {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 100}
Best F1 score (CV): 0.8377777777777778


In [84]:
best_rf = grid.best_estimator_
y_pred = cross_val_predict(best_rf, X, y, cv=5)

print(classification_report(y, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.69      0.76        16
           1       0.78      0.90      0.84        20

    accuracy                           0.81        36
   macro avg       0.81      0.79      0.80        36
weighted avg       0.81      0.81      0.80        36

Confusion Matrix:
 [[11  5]
 [ 2 18]]
